In [5]:
import pandas as pd
import numpy as np
import dask.array as da
from dask_image.ndfilters import generic_filter as d_gf
from scipy.stats import skew
import datetime    
import pickle
import matplotlib.pyplot as plt
from Functions import feature_creation
from Functions import general_functions

In [6]:
features_path = "../../../features/features/"
output_path ="../../../features/"

### Function to load raw features for a zone and produce all the features

In [13]:

def load_raw(zone_number, chunk_size=(800,800)):
    
    labels = np.load(f"{features_path}/Labels3m_{zone_number}.npy")
    hpmf = np.load(f"{features_path}/HPMF_{zone_number}.npy")
    skyview = np.load(f"{features_path}/SkyViewFactor_{zone_number}.npy")
    impoundment = np.load(f"{features_path}/Impoundment_{zone_number}.npy")
    slope = np.load(f"{features_path}/Slope_{zone_number}.npy")
    DEM = np.load(f"{features_path}/DEM_{zone_number}.npy")
    data_frame = pd.DataFrame({"label_3m":      labels.reshape(-1),
                               "hpmf_raw" :      hpmf.reshape(-1), 
                               "skyview_raw":    skyview.reshape(-1),
                               "impoundment_raw": impoundment.reshape(-1),
                               "slope_raw":      slope.reshape(-1)
                              })
    
    try:
        data_frame = pickle.load(open(f"{output_path}zone_{zone_number}.pickle", "rb"))
        data_frame["DEM_raw"] = DEM.reshape(-1)
    except (OSError, IOError) as e:
        pass
    
    
    #Generating impoundment stream amplification to use with several features.
    streamAmp = feature_creation.streamAmplification(impoundment)
    
    print("starting-DEM:        ", datetime.datetime.now())
    #DEM
    #Manually made
    if "DEM_ditch_detection" not in data_frame.columns:
        data_frame["DEM_ditch_detection"] = feature_creation.DEMDitchDetection(DEM).reshape(-1)
    if "DEM_ditch_detection_no_streams" not in data_frame.columns:
        data_frame["DEM_ditch_detection_no_streams"] = feature_creation.impoundmentDEMStreamRemoval(data_frame["DEM_ditch_detection"].values.reshape((2997,2620)), streamAmp).reshape(-1)
    
    print("starting-skyview:    ", datetime.datetime.now())
    
    # Skyview
    # Manually made
    if "conic_mean" not in data_frame.columns:
        data_frame["conic_mean"] = feature_creation.conicMean(skyview, 5 , 0.975).reshape(-1)
    if "skyview_non_ditch" not in data_frame.columns:
        data_frame["skyview_non_ditch"] = feature_creation.skyViewNonDitchAmplification(skyview).reshape(-1)
    if "skyview_gabor" not in data_frame.columns:
        data_frame["skyview_gabor"] = feature_creation.skyViewGabor(skyview).reshape(-1)
    if "conic_mean_no_streams" not in data_frame.columns:
        data_frame["conic_mean_no_streams"] = feature_creation.skyViewHPMFGaborStreamRemoval(data_frame["conic_mean"].values.reshape((2997,2620)), streamAmp).reshape(-1)
    if "skyview_gabor_no_streams" not in data_frame.columns:
        data_frame["skyview_gabor_no_streams"] = feature_creation.skyViewHPMFGaborStreamRemoval(data_frame["skyview_gabor"].values.reshape((2997,2620)), streamAmp).reshape(-1)
    
    # Mass produced
    
    if "skyview_mean_2" not in data_frame.columns:
        data_frame["skyview_mean_2"] = d_gf(da.from_array(skyview, chunks=chunk_size), np.nanmean, footprint=general_functions.create_circular_mask(2)).compute(scheduler='processes').reshape(-1)
    if "skyview_mean_3" not in data_frame.columns:
        data_frame["skyview_mean_3"] = d_gf(da.from_array(skyview, chunks=chunk_size), np.nanmean, footprint=general_functions.create_circular_mask(3)).compute(scheduler='processes').reshape(-1)
    if "skyview_mean_4" not in data_frame.columns:
        data_frame["skyview_mean_4"] = d_gf(da.from_array(skyview, chunks=chunk_size), np.nanmean, footprint=general_functions.create_circular_mask(4)).compute(scheduler='processes').reshape(-1)
    if "skyview_mean_6" not in data_frame.columns:
        data_frame["skyview_mean_6"] = d_gf(da.from_array(skyview, chunks=chunk_size), np.nanmean, footprint=general_functions.create_circular_mask(6)).compute(scheduler='processes').reshape(-1)
    
    if "skyview_median_2" not in data_frame.columns:
        data_frame["skyview_median_2"] = d_gf(da.from_array(skyview, chunks=chunk_size), np.nanmedian, footprint=general_functions.create_circular_mask(2)).compute(scheduler='processes').reshape(-1)
    if "skyview_median_4" not in data_frame.columns:
        data_frame["skyview_median_4"] = d_gf(da.from_array(skyview, chunks=chunk_size), np.nanmedian, footprint=general_functions.create_circular_mask(4)).compute(scheduler='processes').reshape(-1)
    if "skyview_median_6" not in data_frame.columns:        
        data_frame["skyview_median_6"] = d_gf(da.from_array(skyview, chunks=chunk_size), np.nanmedian, footprint=general_functions.create_circular_mask(6)).compute(scheduler='processes').reshape(-1)
        
    
    if "skyview_min_2" not in data_frame.columns:        
        data_frame["skyview_min_2"] = d_gf(da.from_array(skyview, chunks=chunk_size), np.amin, footprint=general_functions.create_circular_mask(2)).compute(scheduler='processes').reshape(-1)
    if "skyview_min_4" not in data_frame.columns:        
        data_frame["skyview_min_4"] = d_gf(da.from_array(skyview, chunks=chunk_size), np.amin, footprint=general_functions.create_circular_mask(4)).compute(scheduler='processes').reshape(-1)
    if "skyview_min_6" not in data_frame.columns:        
        data_frame["skyview_min_6"] = d_gf(da.from_array(skyview, chunks=chunk_size), np.amin, footprint=general_functions.create_circular_mask(6)).compute(scheduler='processes').reshape(-1)        
        
    if "skyview_max_2" not in data_frame.columns:        
        data_frame["skyview_max_2"] = d_gf(da.from_array(skyview, chunks=chunk_size), np.amax, footprint=general_functions.create_circular_mask(2)).compute(scheduler='processes').reshape(-1)
    if "skyview_max_4" not in data_frame.columns:        
        data_frame["skyview_max_4"] = d_gf(da.from_array(skyview, chunks=chunk_size), np.amax, footprint=general_functions.create_circular_mask(4)).compute(scheduler='processes').reshape(-1)
    if "skyview_max_6" not in data_frame.columns:        
        data_frame["skyview_max_6"] = d_gf(da.from_array(skyview, chunks=chunk_size), np.amax, footprint=general_functions.create_circular_mask(6)).compute(scheduler='processes').reshape(-1)        
    
    
    if "skyview_std_2" not in data_frame.columns:        
        data_frame["skyview_std_2"] = d_gf(da.from_array(skyview, chunks=chunk_size), np.nanstd, footprint=general_functions.create_circular_mask(2)).compute(scheduler='processes').reshape(-1)
    if "skyview_std_4" not in data_frame.columns:        
        data_frame["skyview_std_4"] = d_gf(da.from_array(skyview, chunks=chunk_size), np.nanstd, footprint=general_functions.create_circular_mask(4)).compute(scheduler='processes').reshape(-1)
    if "skyview_std_6" not in data_frame.columns:        
        data_frame["skyview_std_6"] = d_gf(da.from_array(skyview, chunks=chunk_size), np.nanstd, footprint=general_functions.create_circular_mask(6)).compute(scheduler='processes').reshape(-1)
        
    if "skyview_skew_2" not in data_frame.columns:        
        data_frame["skyview_skew_2"] = d_gf(da.from_array(skyview, chunks=chunk_size), skew, footprint=general_functions.create_circular_mask(2)).compute(scheduler='processes').reshape(-1)
    
    
    
    print("starting-impoundment:", datetime.datetime.now())
    
    # Impoundment
    # Manually made
    if "impoundment_amplified" not in data_frame.columns:
        data_frame["impoundment_amplified"]= feature_creation.impoundmentAmplification(impoundment).reshape(-1)
    if "impoundment_amplified_no_streams" not in data_frame.columns:
        data_frame["impoundment_amplified_no_streams"] = feature_creation.impoundmentDEMStreamRemoval(data_frame["impoundment_amplified"].values.reshape((2997,2620)), streamAmp).reshape(-1)

    
    # Mass produced
    if "impoundment_mean_2" not in data_frame.columns:
        data_frame["impoundment_mean_2"] = d_gf(da.from_array(impoundment, chunks=chunk_size), np.nanmean, footprint=general_functions.create_circular_mask(2)).compute(scheduler='processes').reshape(-1)
    if "impoundment_mean_3" not in data_frame.columns:
        data_frame["impoundment_mean_3"] = d_gf(da.from_array(impoundment, chunks=chunk_size), np.nanmean, footprint=general_functions.create_circular_mask(3)).compute(scheduler='processes').reshape(-1)
    if "impoundment_mean_4" not in data_frame.columns:
        data_frame["impoundment_mean_4"] = d_gf(da.from_array(impoundment, chunks=chunk_size), np.nanmean, footprint=general_functions.create_circular_mask(4)).compute(scheduler='processes').reshape(-1)
    if "impoundment_mean_6" not in data_frame.columns:
        data_frame["impoundment_mean_6"] = d_gf(da.from_array(impoundment, chunks=chunk_size), np.nanmean, footprint=general_functions.create_circular_mask(6)).compute(scheduler='processes').reshape(-1)
    
    if "impoundment_median_2" not in data_frame.columns:
        data_frame["impoundment_median_2"] = d_gf(da.from_array(impoundment, chunks=chunk_size), np.nanmedian, footprint=general_functions.create_circular_mask(2)).compute(scheduler='processes').reshape(-1)
    if "impoundment_median_4" not in data_frame.columns:
        data_frame["impoundment_median_4"] = d_gf(da.from_array(impoundment, chunks=chunk_size), np.nanmedian, footprint=general_functions.create_circular_mask(4)).compute(scheduler='processes').reshape(-1)
    if "impoundment_median_6" not in data_frame.columns:
        data_frame["impoundment_median_6"] = d_gf(da.from_array(impoundment, chunks=chunk_size), np.nanmedian, footprint=general_functions.create_circular_mask(6)).compute(scheduler='processes').reshape(-1)
    
    if "impoundment_min_2" not in data_frame.columns:        
        data_frame["impoundment_min_2"] = d_gf(da.from_array(impoundment, chunks=chunk_size), np.amin, footprint=general_functions.create_circular_mask(2)).compute(scheduler='processes').reshape(-1)
    if "impoundment_min_4" not in data_frame.columns:        
        data_frame["impoundment_min_4"] = d_gf(da.from_array(impoundment, chunks=chunk_size), np.amin, footprint=general_functions.create_circular_mask(4)).compute(scheduler='processes').reshape(-1)
    if "impoundment_min_6" not in data_frame.columns:        
        data_frame["impoundment_min_6"] = d_gf(da.from_array(impoundment, chunks=chunk_size), np.amin, footprint=general_functions.create_circular_mask(6)).compute(scheduler='processes').reshape(-1)
    
    if "impoundment_max_2" not in data_frame.columns:        
        data_frame["impoundment_max_2"] = d_gf(da.from_array(impoundment, chunks=chunk_size), np.amax, footprint=general_functions.create_circular_mask(2)).compute(scheduler='processes').reshape(-1)
    if "impoundment_max_4" not in data_frame.columns:        
        data_frame["impoundment_max_4"] = d_gf(da.from_array(impoundment, chunks=chunk_size), np.amax, footprint=general_functions.create_circular_mask(4)).compute(scheduler='processes').reshape(-1)
    if "impoundment_max_6" not in data_frame.columns:        
        data_frame["impoundment_max_6"] = d_gf(da.from_array(impoundment, chunks=chunk_size), np.amax, footprint=general_functions.create_circular_mask(6)).compute(scheduler='processes').reshape(-1)
    
    
    if "impoundment_std_2" not in data_frame.columns:
        data_frame["impoundment_std_2"] = d_gf(da.from_array(impoundment, chunks=chunk_size), np.nanstd, footprint=general_functions.create_circular_mask(2)).compute(scheduler='processes').reshape(-1)
    if "impoundment_std_4" not in data_frame.columns:
        data_frame["impoundment_std_4"] = d_gf(da.from_array(impoundment, chunks=chunk_size), np.nanstd, footprint=general_functions.create_circular_mask(4)).compute(scheduler='processes').reshape(-1)
    if "impoundment_std_6" not in data_frame.columns:
        data_frame["impoundment_std_6"] = d_gf(da.from_array(impoundment, chunks=chunk_size), np.nanstd, footprint=general_functions.create_circular_mask(6)).compute(scheduler='processes').reshape(-1)
    
    if "impoundment_skew_2" not in data_frame.columns:
        data_frame["impoundment_skew_2"] = d_gf(da.from_array(impoundment, chunks=chunk_size), skew, footprint=general_functions.create_circular_mask(2)).compute(scheduler='processes').reshape(-1)
    
    
    print("starting-hpmf:       ", datetime.datetime.now())
    # Hpmf
    # Manually made
    if "hpmf_filter" not in data_frame.columns:
        data_frame["hpmf_filter"] = feature_creation.hpmfFilter(hpmf).reshape(-1)
    if "hpmf_gabor" not in data_frame.columns:
        data_frame["hpmf_gabor"] = feature_creation.skyViewGabor(hpmf).reshape(-1)
    if "hpmf_gabor_no_streams" not in data_frame.columns:
        data_frame["hpmf_gabor_no_streams"] = feature_creation.skyViewHPMFGaborStreamRemoval(data_frame["hpmf_gabor"].values.reshape((2997,2620)), streamAmp).reshape(-1)
    if "hpmf_filter_no_streams" not in data_frame.columns:
        data_frame["hpmf_filter_no_streams"] = feature_creation.skyViewHPMFGaborStreamRemoval(feature_creation.hpmfFilter(hpmf).compute(scheduler='processes'), streamAmp).reshape(-1)

    
    # Mass produced
    if "hpmf_mean_2" not in data_frame.columns:
        data_frame["hpmf_mean_2"] = d_gf(da.from_array(hpmf, chunks=chunk_size), np.nanmean, footprint=general_functions.create_circular_mask(2)).compute(scheduler='processes').reshape(-1)
    if "hpmf_mean_3" not in data_frame.columns:
        data_frame["hpmf_mean_3"] = d_gf(da.from_array(hpmf, chunks=chunk_size), np.nanmean, footprint=general_functions.create_circular_mask(3)).compute(scheduler='processes').reshape(-1)
    if "hpmf_mean_4" not in data_frame.columns:
        data_frame["hpmf_mean_4"] = d_gf(da.from_array(hpmf, chunks=chunk_size), np.nanmean, footprint=general_functions.create_circular_mask(4)).compute(scheduler='processes').reshape(-1)
    if "hpmf_mean_6" not in data_frame.columns:
        data_frame["hpmf_mean_6"] = d_gf(da.from_array(hpmf, chunks=chunk_size), np.nanmean, footprint=general_functions.create_circular_mask(6)).compute(scheduler='processes').reshape(-1)
    
    if "hpmf_median_2" not in data_frame.columns:
        data_frame["hpmf_median_2"] = d_gf(da.from_array(hpmf, chunks=chunk_size), np.nanmedian, footprint=general_functions.create_circular_mask(2)).compute(scheduler='processes').reshape(-1)
    if "hpmf_median_4" not in data_frame.columns:
        data_frame["hpmf_median_4"] = d_gf(da.from_array(hpmf, chunks=chunk_size), np.nanmedian, footprint=general_functions.create_circular_mask(4)).compute(scheduler='processes').reshape(-1)
    if "hpmf_median_6" not in data_frame.columns:
        data_frame["hpmf_median_6"] = d_gf(da.from_array(hpmf, chunks=chunk_size), np.nanmedian, footprint=general_functions.create_circular_mask(6)).compute(scheduler='processes').reshape(-1)
    
    if "hpmf_min_2" not in data_frame.columns:        
        data_frame["hpmf_min_2"] = d_gf(da.from_array(hpmf, chunks=chunk_size), np.amin, footprint=general_functions.create_circular_mask(2)).compute(scheduler='processes').reshape(-1)
    if "hpmf_min_4" not in data_frame.columns:        
        data_frame["hpmf_min_4"] = d_gf(da.from_array(hpmf, chunks=chunk_size), np.amin, footprint=general_functions.create_circular_mask(4)).compute(scheduler='processes').reshape(-1)
    if "hpmf_min_6" not in data_frame.columns:        
        data_frame["hpmf_min_6"] = d_gf(da.from_array(hpmf, chunks=chunk_size), np.amin, footprint=general_functions.create_circular_mask(6)).compute(scheduler='processes').reshape(-1)
    
    if "hpmf_max_2" not in data_frame.columns:        
        data_frame["hpmf_max_2"] = d_gf(da.from_array(hpmf, chunks=chunk_size), np.amax, footprint=general_functions.create_circular_mask(2)).compute(scheduler='processes').reshape(-1)
    if "hpmf_max_4" not in data_frame.columns:        
        data_frame["hpmf_max_4"] = d_gf(da.from_array(hpmf, chunks=chunk_size), np.amax, footprint=general_functions.create_circular_mask(4)).compute(scheduler='processes').reshape(-1)
    if "hpmf_max_6" not in data_frame.columns:        
        data_frame["hpmf_max_6"] = d_gf(da.from_array(hpmf, chunks=chunk_size), np.amax, footprint=general_functions.create_circular_mask(6)).compute(scheduler='processes').reshape(-1)
    
    
    if "hpmf_std_2" not in data_frame.columns:
        data_frame["hpmf_std_2"] = d_gf(da.from_array(hpmf, chunks=chunk_size), np.nanstd, footprint=general_functions.create_circular_mask(2)).compute(scheduler='processes').reshape(-1)
    if "hpmf_std_4" not in data_frame.columns:
        data_frame["hpmf_std_4"] = d_gf(da.from_array(hpmf, chunks=chunk_size), np.nanstd, footprint=general_functions.create_circular_mask(4)).compute(scheduler='processes').reshape(-1)
    if "hpmf_std_6" not in data_frame.columns:
        data_frame["hpmf_std_6"] = d_gf(da.from_array(hpmf, chunks=chunk_size), np.nanstd, footprint=general_functions.create_circular_mask(6)).compute(scheduler='processes').reshape(-1)
    
    if "hpmf_skew_2" not in data_frame.columns:
        data_frame["hpmf_skew_2"] = d_gf(da.from_array(hpmf, chunks=chunk_size), skew, footprint=general_functions.create_circular_mask(2)).compute(scheduler='processes').reshape(-1)
    
    print("starting-slope:      ", datetime.datetime.now())
    # Slope
    # Manually made
    if "slope_non_ditch" not in data_frame.columns:
        data_frame["slope_non_ditch"] = feature_creation.slopeNonDitchAmplification(slope).reshape(-1)
    
    # Mass produced
    if "slope_mean_2" not in data_frame.columns:
        data_frame["slope_mean_2"] = d_gf(da.from_array(slope, chunks=chunk_size), np.nanmean, footprint=general_functions.create_circular_mask(2)).compute(scheduler='processes').reshape(-1)
    if "slope_mean_3" not in data_frame.columns:
        data_frame["slope_mean_3"] = d_gf(da.from_array(slope, chunks=chunk_size), np.nanmean, footprint=general_functions.create_circular_mask(3)).compute(scheduler='processes').reshape(-1)
    if "slope_mean_4" not in data_frame.columns:
        data_frame["slope_mean_4"] = d_gf(da.from_array(slope, chunks=chunk_size), np.nanmean, footprint=general_functions.create_circular_mask(4)).compute(scheduler='processes').reshape(-1)
    if "slope_mean_6" not in data_frame.columns:
        data_frame["slope_mean_6"] = d_gf(da.from_array(slope, chunks=chunk_size), np.nanmean, footprint=general_functions.create_circular_mask(6)).compute(scheduler='processes').reshape(-1)
    
    if "slope_median_2" not in data_frame.columns:
        data_frame["slope_median_2"] = d_gf(da.from_array(slope, chunks=chunk_size), np.nanmedian, footprint=general_functions.create_circular_mask(2)).compute(scheduler='processes').reshape(-1)
    if "slope_median_4" not in data_frame.columns:
        data_frame["slope_median_4"] = d_gf(da.from_array(slope, chunks=chunk_size), np.nanmedian, footprint=general_functions.create_circular_mask(4)).compute(scheduler='processes').reshape(-1)
    if "slope_median_6" not in data_frame.columns:
        data_frame["slope_median_6"] = d_gf(da.from_array(slope, chunks=chunk_size), np.nanmedian, footprint=general_functions.create_circular_mask(6)).compute(scheduler='processes').reshape(-1)
    
    if "slope_min_2" not in data_frame.columns:        
        data_frame["slope_min_2"] = d_gf(da.from_array(slope, chunks=chunk_size), np.amin, footprint=general_functions.create_circular_mask(2)).compute(scheduler='processes').reshape(-1)
    if "slope_min_4" not in data_frame.columns:        
        data_frame["slope_min_4"] = d_gf(da.from_array(slope, chunks=chunk_size), np.amin, footprint=general_functions.create_circular_mask(4)).compute(scheduler='processes').reshape(-1)
    if "slope_min_6" not in data_frame.columns:        
        data_frame["slope_min_6"] = d_gf(da.from_array(slope, chunks=chunk_size), np.amin, footprint=general_functions.create_circular_mask(6)).compute(scheduler='processes').reshape(-1)
    
    if "slope_max_2" not in data_frame.columns:        
        data_frame["slope_max_2"] = d_gf(da.from_array(slope, chunks=chunk_size), np.amax, footprint=general_functions.create_circular_mask(2)).compute(scheduler='processes').reshape(-1)
    if "slope_max_4" not in data_frame.columns:        
        data_frame["slope_max_4"] = d_gf(da.from_array(slope, chunks=chunk_size), np.amax, footprint=general_functions.create_circular_mask(4)).compute(scheduler='processes').reshape(-1)
    if "hpmf_max_6" not in data_frame.columns:        
        data_frame["slope_max_6"] = d_gf(da.from_array(slope, chunks=chunk_size), np.amax, footprint=general_functions.create_circular_mask(6)).compute(scheduler='processes').reshape(-1)
        
    if "slope_std_2" not in data_frame.columns:
        data_frame["slope_std_2"] = d_gf(da.from_array(slope, chunks=chunk_size), np.nanstd, footprint=general_functions.create_circular_mask(2)).compute(scheduler='processes').reshape(-1)
    if "slope_std_4" not in data_frame.columns:
        data_frame["slope_std_4"] = d_gf(da.from_array(slope, chunks=chunk_size), np.nanstd, footprint=general_functions.create_circular_mask(4)).compute(scheduler='processes').reshape(-1)
    if "slope_std_6" not in data_frame.columns:
        data_frame["slope_std_6"] = d_gf(da.from_array(slope, chunks=chunk_size), np.nanstd, footprint=general_functions.create_circular_mask(6)).compute(scheduler='processes').reshape(-1)
    
    if "slope_skew_2" not in data_frame.columns:
        data_frame["slope_skew_2"] = d_gf(da.from_array(slope, chunks=chunk_size), skew, footprint=general_functions.create_circular_mask(2)).compute(scheduler='processes').reshape(-1)
    
    return data_frame

def set_order(dataframe):
    return dataframe[[
    'label_3m',
    'hpmf_raw',
    'skyview_raw',
    'impoundment_raw',
    'slope_raw',
    'DEM_ditch_detection',
    'DEM_ditch_detection_no_streams',
    'conic_mean',
    'skyview_non_ditch',
    'skyview_gabor',
    'conic_mean_no_streams',
    'skyview_gabor_no_streams',
    'skyview_mean_2',
    'skyview_mean_3',
    'skyview_mean_4',
    'skyview_mean_6',
    'skyview_median_2',
    'skyview_median_4',
    'skyview_median_6',
    'skyview_min_2',
    'skyview_min_4',
    'skyview_min_6',
    'skyview_max_2',
    'skyview_max_4',
    'skyview_max_6',
    'skyview_std_2',
    'skyview_std_4',
    'skyview_std_6',
    'impoundment_amplified',
    'impoundment_amplified_no_streams',
    'impoundment_mean_2',
    'impoundment_mean_3',
    'impoundment_mean_4',
    'impoundment_mean_6',
    'impoundment_median_2',
    'impoundment_median_4',
    'impoundment_median_6',
    'impoundment_min_2',
    'impoundment_min_4',
    'impoundment_min_6',
    'impoundment_max_2',
    'impoundment_max_4',
    'impoundment_max_6',
    'impoundment_std_2',
    'impoundment_std_4',
    'impoundment_std_6',
    'hpmf_filter',
    'hpmf_gabor',
    'hpmf_gabor_no_streams',
    'hpmf_filter_no_streams',
    'hpmf_mean_2',
    'hpmf_mean_3',
    'hpmf_mean_4',
    'hpmf_mean_6',
    'hpmf_median_2',
    'hpmf_median_4',
    'hpmf_median_6',
    'hpmf_min_2',
    'hpmf_min_4',
    'hpmf_min_6',
    'hpmf_max_2',
    'hpmf_max_4',
    'hpmf_max_6',
    'hpmf_std_2',
    'hpmf_std_4',
    'hpmf_std_6',
    'slope_non_ditch',
    'slope_mean_2',
    'slope_mean_3',
    'slope_mean_4',
    'slope_mean_6',
    'slope_median_2',
    'slope_median_4',
    'slope_median_6',
    'slope_min_2',
    'slope_min_4',
    'slope_min_6',
    'slope_max_2',
    'slope_max_4',
    'hpmf_max_6',
    'slope_std_2',
    'slope_std_4',
    'slope_std_6',
    ]]

### Create the features for all 21 zones

In [14]:
%%time
zone_with_features = load_raw(1, (900, 875))
zone_with_features = set_order(zone_with_features)
zone_with_features.to_pickle(f"{output_path}zone_1.pickle")

starting-DEM: 2019-04-17 17:12:24.952044
starting-skyview: 2019-04-17 17:12:24.952544
starting-impoundment: 2019-04-17 17:15:23.638389
starting-hpmf: 2019-04-17 17:18:25.762480
starting-slope: 2019-04-17 17:21:26.505497
CPU times: user 3min 1s, sys: 1min 23s, total: 4min 25s
Wall time: 13min 42s


In [16]:
%%time
zone_with_features = load_raw(2, (900, 875))
zone_with_features = set_order(zone_with_features)
zone_with_features.to_pickle(f"{output_path}zone_2.pickle")

starting-DEM: 2019-04-17 17:27:02.557300
starting-skyview: 2019-04-17 17:27:02.557462
starting-impoundment: 2019-04-17 17:30:11.555352
starting-hpmf: 2019-04-17 17:33:16.204204
starting-slope: 2019-04-17 17:36:27.323132
CPU times: user 2min 59s, sys: 1min 39s, total: 4min 38s
Wall time: 14min 4s


In [17]:
%%time
zone_with_features = load_raw(3, (900, 875))
zone_with_features = set_order(zone_with_features)
zone_with_features.to_pickle(f"{output_path}zone_3.pickle")

starting-DEM: 2019-04-17 17:41:14.808052
starting-skyview: 2019-04-17 17:41:14.808197
starting-impoundment: 2019-04-17 17:44:20.977283
starting-hpmf: 2019-04-17 17:47:29.045736
starting-slope: 2019-04-17 17:50:35.129400
CPU times: user 2min 57s, sys: 1min 39s, total: 4min 36s
Wall time: 14min 8s


In [18]:
%%time
zone_with_features = load_raw(4, (900, 875))
zone_with_features = set_order(zone_with_features)
zone_with_features.to_pickle(f"{output_path}zone_4.pickle")

starting-DEM: 2019-04-17 17:55:22.677541
starting-skyview: 2019-04-17 17:55:22.677685
starting-impoundment: 2019-04-17 17:58:29.505347
starting-hpmf: 2019-04-17 18:01:34.218186
starting-slope: 2019-04-17 18:04:38.578061
CPU times: user 3min, sys: 1min 39s, total: 4min 39s
Wall time: 14min 9s


In [19]:
%%time
zone_with_features = load_raw(5, (900, 875))
zone_with_features = set_order(zone_with_features)
zone_with_features.to_pickle(f"{output_path}zone_5.pickle")

starting-DEM: 2019-04-17 18:09:37.481308
starting-skyview: 2019-04-17 18:09:37.481619
starting-impoundment: 2019-04-17 18:12:39.980739
starting-hpmf: 2019-04-17 18:15:43.931996
starting-slope: 2019-04-17 18:18:49.931544
CPU times: user 3min 3s, sys: 1min 39s, total: 4min 43s
Wall time: 14min 13s


In [20]:
%%time
zone_with_features = load_raw(6, (900, 875))
zone_with_features = set_order(zone_with_features)
zone_with_features.to_pickle(f"{output_path}zone_6.pickle")

starting-DEM: 2019-04-17 18:23:47.093151
starting-skyview: 2019-04-17 18:23:47.093648
starting-impoundment: 2019-04-17 18:26:52.869664
starting-hpmf: 2019-04-17 18:29:58.671928
starting-slope: 2019-04-17 18:33:03.772903
CPU times: user 2min 59s, sys: 1min 38s, total: 4min 37s
Wall time: 14min 5s


In [21]:
%%time
zone_with_features = load_raw(7, (900, 875))
zone_with_features = set_order(zone_with_features)
zone_with_features.to_pickle(f"{output_path}zone_7.pickle")

starting-DEM: 2019-04-17 18:37:51.534515
starting-skyview: 2019-04-17 18:37:51.535016
starting-impoundment: 2019-04-17 18:40:55.803009
starting-hpmf: 2019-04-17 18:44:00.477388
starting-slope: 2019-04-17 18:47:15.901240
CPU times: user 2min 59s, sys: 1min 39s, total: 4min 38s
Wall time: 14min 12s


In [ ]:
%%time
zone_with_features = load_raw(8, (900, 875))
zone_with_features.to_pickle(f"{output_path}zone_8.pickle")
zone_with_features = None

In [23]:
%%time
zone_with_features = load_raw(9, (900, 875))
zone_with_features = set_order(zone_with_features)
zone_with_features.to_pickle(f"{output_path}zone_9.pickle")

starting-DEM: 2019-04-17 19:57:58.910450
starting-skyview: 2019-04-17 19:57:58.910662
starting-impoundment: 2019-04-17 20:03:14.311256
starting-hpmf: 2019-04-17 20:06:22.220674
starting-slope: 2019-04-17 20:09:34.124697
CPU times: user 3min 19s, sys: 1min 50s, total: 5min 10s
Wall time: 16min 44s


In [ ]:
%%time
zone_with_features = load_raw(10, (900, 875))
zone_with_features.to_pickle(f"{output_path}zone_10.pickle")
zone_with_features = None

In [ ]:
%%time
zone_with_features = load_raw(11, (900, 875))
zone_with_features.to_pickle(f"{output_path}zone_11.pickle")
zone_with_features = None